In [0]:
%sql
use catalog customersdata;
use schema gold;

###Read data from silver tables

In [0]:
from pyspark.sql import functions as F

df_customers_silver = spark.table('customersdata.silver.customers_silver')
df_orders_silver = spark.table('customersdata.silver.orders_silver')

###Join customers and orders table

In [0]:
df_joined_gold = (
  df_orders_silver
    .join(df_customers_silver, on='customer_id', how='left')
)

display(df_joined_gold)

#save this in gold table

df_joined_gold.write.mode('overwrite').saveAsTable('customersdata.gold.joined_gold')

###Revenue by country and segment

In [0]:
df_country_segment = (
    df_joined_gold
      .groupBy("country", "segment")
      .agg(
          F.sum("net_amount").alias("total_revenue"),
          F.countDistinct("customer_id").alias("unique_customers"),
          F.count("order_id").alias("num_orders")
      )
)

display(df_country_segment)

df_country_segment.write.mode("overwrite").saveAsTable("customersdata.gold.revenue_by_country_segment")


###Top products

In [0]:
df_product_sales = (
    df_joined_gold
      .groupBy("product", "category")
      .agg(
          F.sum("net_amount").alias("total_revenue"),
          F.sum("quantity").alias("total_quantity"),
          F.countDistinct("customer_id").alias("num_customers")
      )
)

display(df_product_sales)

df_product_sales.write.mode("overwrite").saveAsTable("customersdata.gold.product_sales_summary")
